In [73]:
from sklearn.model_selection import train_test_split
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import(ModelSpec as MS,summarize)

In [74]:
#5(a)
Default = load_data('Default')
Default

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [75]:
# Check for missing data
assert Default.isnull().sum().sum() == 0

In [104]:
#(a)Fit a logistic regression model using income and balance to predict default
allvars = Default.columns.drop(['default','student'])
design = MS(allvars)
X=design.fit_transform(Default)
y=Default.default == 'Yes'
glm = sm.GLM(y,X,family=sm.families.Binomial())
results=glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-11.540500,0.435000,-26.544,0.0
balance,0.005600,0.000000,24.835,0.0
income,0.000021,0.000005,4.174,0.0


In [105]:
probs=results.predict()
probs[:10]

array([1.50472802e-03, 1.26192994e-03, 8.02621054e-03, 4.05995722e-04,
       1.82672370e-03, 2.04243978e-03, 1.72559300e-03, 1.34825838e-03,
       1.47134060e-02, 1.78894752e-05])

In [99]:
#(b)
#i.Split the sample set into a training set and a validation set 
Default_train, Default_valid = train_test_split(Default,
                                                test_size=5000, 
                                                random_state=0)

In [102]:
#ii.Fit logistic regression model using only training observations

hp_mm = MS(['balance','income'])
X_train=hp_mm.fit_transform(Default_train)
y_train=Default_train.default == 'Yes'
glm = sm.GLM(y,X,family=sm.families.Binomial())
results=glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-12.461000,0.95300,-13.079,0.000
balance,0.006100,0.00000,12.368,0.000
income,0.000019,0.00001,1.890,0.059


In [103]:


X_valid = hp_mm.transform(Default_valid)  # 用于验证集的转换方式与训练集相同
y_valid = Default_valid.default == 'Yes'  # 确保目标变量的处理与训练集一致
probs_valid = results.predict(X_valid)
labels_valid = np.where(probs_valid > 0.5, 'Yes', 'No')
validation_error = np.mean(labels_valid != y_valid)
validation_error

1.0

In [93]:
#iii.Prediction of default ststus in validation set
probs=results.predict(Default_valid)
probs[:10]

ValueError: shapes (7000,4) and (3,) not aligned: 4 (dim 1) != 3 (dim 0)

In [80]:
labels = np.where(probs > 0.5, 'Yes', 'No')  
confusion_table(labels, Default_train.default)

Truth,No,Yes
Predicted,,
No,4834,101
Yes,19,46


In [81]:
np.mean(labels==Default_train.default),(4834+46)/5000

(0.976, 0.976)

In [82]:
#iv.Validation set error:0.024
np.mean(labels != Default_train.default)

0.024

In [86]:
#(c)
#1.test_size=5000
Default_train, Default_valid = train_test_split(Default,
                                                test_size=3000, 
                                                random_state=0)
allvars1 = Default_train.columns.drop(['default','student'])
design1 = MS(allvars1)
X=design1.fit_transform(Default_train)
y=Default_train.default == 'Yes'
glm = sm.GLM(y,X,family=sm.families.Binomial())
results=glm.fit()
summarize(results)
probs=results.predict()
labels = np.where(probs > 0.5, 'Yes', 'No')  
confusion_table(labels, Default_train.default)
np.mean(labels != Default_train.default)

0.025857142857142856

In [88]:
#1.test_size=1000
Default_train, Default_valid = train_test_split(Default,
                                                test_size=1000, 
                                                random_state=0)
allvars1 = Default_train.columns.drop(['default','student'])
design1 = MS(allvars1)
X=design1.fit_transform(Default_train)
y=Default_train.default == 'Yes'
glm = sm.GLM(y,X,family=sm.families.Binomial())
results=glm.fit()
summarize(results)
probs=results.predict()
labels = np.where(probs > 0.5, 'Yes', 'No')  
confusion_table(labels, Default_train.default)
np.mean(labels != Default_train.default)

0.02588888888888889

In [89]:
#1.test_size=7000
Default_train, Default_valid = train_test_split(Default,
                                                test_size=7000, 
                                                random_state=0)
allvars1 = Default_train.columns.drop(['default','student'])
design1 = MS(allvars1)
X=design1.fit_transform(Default_train)
y=Default_train.default == 'Yes'
glm = sm.GLM(y,X,family=sm.families.Binomial())
results=glm.fit()
summarize(results)
probs=results.predict()
labels = np.where(probs > 0.5, 'Yes', 'No')  
confusion_table(labels, Default_train.default)
np.mean(labels != Default_train.default)

0.023333333333333334

In [90]:
#(d)Logistic Regression Model with Dummy Variable（‘student’）
X = Default[['income', 'balance', 'student']]
X['student'] = (X['student'] == 'Yes').astype(int)  # Convert 'student' to dummy variable
y = Default['default']

for i in range(3):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.5, random_state=i)
    log_reg = LogisticRegression()
    log_reg.fit(X_train, y_train)
    y_pred = log_reg.predict(X_valid)
    validation_error = np.mean(y_pred != y_valid)
    print(f"Validation Set Error with Dummy Variable {i + 1}:", validation_error)



Validation Set Error with Dummy Variable 1: 0.0356
Validation Set Error with Dummy Variable 2: 0.0322
Validation Set Error with Dummy Variable 3: 0.032


/var/folders/b1/_lbgm2qn7lj0s6f8mq3gk_0h0000gn/T/ipykernel_8009/640627621.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['student'] = (X['student'] == 'Yes').astype(int)  # Convert 'student' to dummy variable
